In [1]:
# Purpose: To extract VIN from image files based on Google Vision API
# Author:  spicyhill 

# Get Google keys from a seperate file.
# How to setup Google API: https://cloud.google.com/vision/docs/setup
%run ./google_key.ipynb
api_key = GOOGLE_API_KEY
credential_path = VISION_KEY

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =credential_path

import io

from google.cloud import vision

# re is used to extract VIN using regex
import re

# display local image
from IPython.display import Image

In [2]:
# Detects VIN in the image file
# index = 0: the file located in Google Cloud Storage or on the Web
# index = 1: the file located in local folder
def detect_vin(path, index):
    client = vision.ImageAnnotatorClient()
    
    if index == 0:
        image = vision.Image()
        image.source.image_uri = path
    elif index == 1: 
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
    else:
        print("Please input correct index number.\n"+
              "0: the file located in Google Cloud Storage or on the Web\n"+
              "1: the file located in local folder")
        return 

    response = client.text_detection(image=image)
    if len(response.text_annotations) != 0:
        texts = response.text_annotations[0].description
    
        # VIN is a 17 digit alphanumeric unique code
        # Use regular expressions to extract VIN
        vin = re.findall('[A-Za-z0-9_]{17}', texts)
        if len(vin) == 0:
            vin = ['0']
    else:
        vin = ['0']

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return vin[0]

## Extract VIN from Vehicle Permit

![title](https://www.sellmycar.ca/wp-content/uploads/2009/02/ontario-vin-number.jpg)

In [6]:
# Vehicle Permit
uri="https://www.sellmycar.ca/wp-content/uploads/2009/02/ontario-vin-number.jpg"
detect_vin(uri, 0)

'5FNRL5H60BB507148'

## Extract VIN from VIN Photo

![title](https://assets.whichcar.com.au/image/upload/s--hqwMg5vO--/ar_2.304921968787515,c_fill,f_auto,q_auto:good/c_scale,w_1536/v1/archive/whichcar/2018/09/14/-1/VIN-MAIN-PIC.jpg)

In [7]:
# VIN 
uri="https://assets.whichcar.com.au/image/upload/s--hqwMg5vO--/ar_2.304921968787515,c_fill,f_auto,q_auto:good/c_scale,w_1536/v1/archive/whichcar/2018/09/14/-1/VIN-MAIN-PIC.jpg"
detect_vin(uri, 0)

'WVGZZZ5NZJM131395'